In [34]:
# !git clone https://github.com/sunsmarterjie/yolov12.git
# %cd yolov12

In [35]:
# %pip install -r requirements.txt
# %pip install -e .

In [1]:
%cd yolov12

/home/himanshu/yolov12


In [2]:
from ultralytics import YOLO

/home/himanshu/python311_env/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FlashAttention is not available on this device. Using scaled_dot_product_attention instead.


In [3]:
model_path = '/home/himanshu/Downloads/face_detector_yolov12.pt'
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO(model_path)
# test_image = '/home/himanshu/Downloads/YoloTestImg.jpeg'
# result = model(test_image)
# result[0].show()


image 1/1 /home/himanshu/Downloads/YoloTestImg.jpeg: 256x320 6 faces, 73.1ms
Speed: 2.8ms preprocess, 73.1ms inference, 3.2ms postprocess per image at shape (1, 3, 256, 320)


In [3]:
# !pip install opencv-python

In [2]:
import cv2
import os
from datetime import datetime
from ultralytics import YOLO  # Make sure ultralytics is installed
import string
import time


# Add at the start of the loop
current_time = time.time()

# Add this outside the loop to store the last time an image was captured
last_capture_time = 0
capture_interval = 0.3  # seconds between captures (e.g., 0.5s = 500ms)


# Load your trained YOLOv12 model
model_path = "/home/himanshu/Downloads/face_detector_yolov12.pt"
model = YOLO(model_path)

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam")
    exit()

# Base directory where data will be saved
dataset_dir = "human_faces_dataset"
os.makedirs(dataset_dir, exist_ok=True)

# Settings
images_per_class = 100
capture_key_map = {str(i): f"human{i}" for i in range(1, 10)}  # '1' → 'horse1', etc.

print("[INFO] Press 1–9 to start capturing for human1 to human9")
print("[INFO] Press 'q' to quit")

current_label = None
image_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect horse faces
        current_time = time.time()
        results = model.predict(source=frame, device='cpu', conf=0.6, verbose=False)
        boxes = results[0].boxes

        # Draw boxes and current capture state
        adjusted_boxes = None
        for box in boxes:
            # x1, y1, x2, y2 = map(int, box.xyxy[0])
            h, w, _ = frame.shape
            x1, y1, x2, y2 = box.xyxy[0].tolist()

            # Expand box: top +5%, left/right +2%
            box_width = x2 - x1
            box_height = y2 - y1
                
            x1 = max(0, int(x1 - 0.04 * box_width))
            x2 = min(w, int(x2 + 0.04 * box_width))
            y1 = max(0, int(y1 - 0.15 * box_height))
            y2 = min(h, int(y2))  # bottom stays same
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            if adjusted_boxes is None:
                adjusted_boxes = (x1 , y1, x2 , y2)

        if current_label and image_count < images_per_class:
            # Save the first detected horse face
            if len(boxes) > 0 and (current_time - last_capture_time) > capture_interval:
                x1, y1, x2, y2 = adjusted_boxes
                face_crop = frame[y1:y2, x1:x2]
                
                label_dir = os.path.join(dataset_dir, current_label)
                os.makedirs(label_dir, exist_ok=True)
                
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                filename = f"{label_dir}/img_{timestamp}.jpg"
                cv2.imwrite(filename, face_crop)
                print(f"[SAVED] {filename}")
                image_count += 1
                last_capture_time = current_time  # update last capture time                

        # Display info
        status_text = f"Capturing: {current_label or 'None'} ({image_count}/{images_per_class})"
        cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 255), 2)

        # Show video
        cv2.imshow("Human Face Collector", frame)

        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            print("Exiting.")
            break

        elif chr(key) in capture_key_map.keys():
            current_label = capture_key_map[chr(key)]
            image_count = 0
            # print(f"[INFO] Started capturing for: {current_label}")

finally:
    cap.release()
    cv2.destroyAllWindows()


[INFO] Press 1–9 to start capturing for human1 to human9
[INFO] Press 'q' to quit
[SAVED] human_faces_dataset/human1/img_20250507_113222_364270.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113222_683675.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113222_993776.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113223_314708.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113223_633203.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113223_952743.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113224_263214.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113224_588006.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113224_903143.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113225_218531.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113225_529604.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113225_842762.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113226_154271.jpg
[SAVED] human_faces_dataset/human1/img_20250507_113226_47030

In [3]:
# ## from torchvision import models, transforms
# import torch.nn as nn
# import torch

# # Load pretrained ResNet50 and remove final classification layer
# resnet = models.resnet50(pretrained=True)
# feature_extractor = nn.Sequential(*list(resnet.children())[:-1])  # Remove FC
# feature_extractor.eval()  # inference mode


In [9]:
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406], 
#         std=[0.229, 0.224, 0.225])
# ])

In [16]:
# import os
# from PIL import Image
# import numpy as np

# def extract_embedding(img_path):
#     image = Image.open(img_path).convert('RGB')
#     image = transform(image).unsqueeze(0)  # Add batch dim
#     with torch.no_grad():
#         embedding = feature_extractor(image).squeeze().numpy()
#         embedding = embedding / np.linalg.norm(embedding)  # normalize
#     return embedding

# # Dictionary to store embeddings
# human_embeddings = {}

# dataset_path = "human_faces_dataset"
# for human_name in os.listdir(dataset_path):
#     class_dir = os.path.join(dataset_path, human_name)
#     if not os.path.isdir(class_dir):
#         continue

#     emb_list = []
#     for img_file in os.listdir(class_dir):
#         img_path = os.path.join(class_dir, img_file)
#         emb = extract_embedding(img_path)
#         emb_list.append(emb)
    
#     # average embedding for that horse
#     human_embeddings[human_name] = np.mean(emb_list, axis=0)


In [20]:
# import os
# import numpy as np
# import pickle

# EMBEDDING_FILE = 'face_embeddings.pkl'

# def save_embeddings(embeddings_dict):
#     with open(EMBEDDING_FILE, 'wb') as f:
#         pickle.dump(human_embeddings, f)


In [45]:
# import cv2
# import torch
# import numpy as np
# from PIL import Image
# from torchvision import models, transforms
# from sklearn.metrics.pairwise import cosine_similarity
# import pickle
# import os

# # ======= SETUP =======
# # Preprocessing transform for ResNet-50
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

# # Load pretrained ResNet-50
# resnet = models.resnet50(pretrained=True)
# resnet.fc = torch.nn.Identity()
# resnet.eval()

# # Load saved embeddings
# with open('face_embeddings.pkl', 'rb') as f:
#     known_embeddings = pickle.load(f)

# # ======= FACE EMBEDDING FUNCTION =======
# def get_face_embedding(face_image):
#     img = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
#     input_tensor = transform(img).unsqueeze(0)
#     with torch.no_grad():
#         embedding = resnet(input_tensor).squeeze(0).numpy()
#         embedding = embedding / np.linalg.norm(embedding)
#     return embedding

# # ======= FACE RECOGNITION FUNCTION =======
# def recognize_face(embedding):
#     best_match = "Unknown"
#     best_score = 0.0
#     for name, stored_emb in known_embeddings.items():
#         score = cosine_similarity([embedding], [stored_emb])[0][0]
#         if score > best_score:
#             best_score = score
#             best_match = name
#     return best_match if best_score > 0.15 else "Unknown"

# # ======= PLACEHOLDER: Your YOLOv12 face detector =======
# # Replace this with actual face detection from YOLOv12
# def detect_faces_yolov12(frame, conf_threshold=0.6):
#     results = model.predict(source=frame, device='cpu', conf=conf_threshold, verbose=False)
#     boxes = results[0].boxes

#     detected_faces = []
#     h, w, _ = frame.shape

#     for box in boxes:
#         x1, y1, x2, y2 = box.xyxy[0].tolist()

#         # Expand box: top +15%, sides +4%
#         box_width = x2 - x1
#         box_height = y2 - y1

#         x1 = max(0, int(x1 - 0.04 * box_width))
#         x2 = min(w, int(x2 + 0.04 * box_width))
#         y1 = max(0, int(y1 - 0.15 * box_height))
#         y2 = min(h, int(y2))  # bottom stays same

#         # Convert to (x, y, w, h)
#         x = x1
#         y = y1
#         w_box = x2 - x1
#         h_box = y2 - y1

#         detected_faces.append((x, y, w_box, h_box))

#     return detected_faces


# # ======= REAL-TIME WEBCAM LOOP =======
# cap = cv2.VideoCapture(0)

# print("Starting face recognition... Press 'q' to quit.")
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Mirror the frame (flip horizontally)
#     frame = cv2.flip(frame, 1)

#     # Detect faces
#     boxes = detect_faces(frame)

#     for (x, y, w, h) in boxes:
#         # Adjust box size (5% top, 2% sides)
#         x1 = int(x - 0.02 * w)
#         y1 = int(y - 0.05 * h)
#         x2 = int(x + w + 0.02 * w)
#         y2 = int(y + h + 0.05 * h)
#         x1, y1 = max(0, x1), max(0, y1)
#         x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)

#         face_crop = frame[y1:y2, x1:x2]
#         emb = get_face_embedding(face_crop)
#         name = recognize_face_binary(emb)

#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
#         cv2.putText(frame, name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

#     cv2.imshow("Face Recognition", frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


/home/himanshu/python311_env/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/himanshu/python311_env/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Starting face recognition... Press 'q' to quit.


FileNotFoundError: [Errno 2] No such file or directory: 'classifiers'

In [43]:
# from sklearn.linear_model import LogisticRegression
# import pickle
# import numpy as np
# import os

# def train_binary_classifiers(embedding_file='face_embeddings.pkl', model_dir='classifiers'):
#     os.makedirs(model_dir, exist_ok=True)
#     with open(embedding_file, 'rb') as f:
#         embeddings = pickle.load(f)

#     names = list(embeddings.keys())
#     X = np.array(list(embeddings.values()))
#     for i, name in enumerate(names):
#         y = np.array([1 if j == i else 0 for j in range(len(names))])
#         clf = LogisticRegression(max_iter=1000)
#         clf.fit(X, y)

#         model_path = os.path.join(model_dir, f"{name}_clf.pkl")
#         with open(model_path, 'wb') as f_out:
#             pickle.dump(clf, f_out)
#         print(f"Saved classifier for {name} to {model_path}")


In [44]:
# def recognize_face_binary(embedding, model_dir='classifiers', threshold=0.6):
#     best_match = "Unknown"
#     best_score = threshold

#     for fname in os.listdir(model_dir):
#         if fname.endswith('_clf.pkl'):
#             name = fname.replace('_clf.pkl', '')
#             with open(os.path.join(model_dir, fname), 'rb') as f:
#                 clf = pickle.load(f)
#             score = clf.predict_proba([embedding])[0][1]
#             if score > best_score:
#                 best_score = score
#                 best_match = name
#     return best_match


In [4]:
# import cv2
# import torch
# import numpy as np
# from PIL import Image
# from torchvision import models, transforms
# from sklearn.linear_model import LogisticRegression
# import pickle
# import os

# # ========== CONFIG ==========
# EMBEDDING_FILE = 'face_embeddings.pkl'
# CLASSIFIER_DIR = 'classifiers'
# THRESHOLD = 0.6  # confidence threshold for binary classification

# # ========== TRANSFORMS & MODEL ==========
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

# resnet = models.resnet50(pretrained=True)
# resnet.fc = torch.nn.Identity()
# resnet.eval()

# # ========== EMBEDDING FUNCTION ==========
# def get_face_embedding(face_image):
#     img = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
#     input_tensor = transform(img).unsqueeze(0)
#     with torch.no_grad():
#         embedding = resnet(input_tensor).squeeze(0).numpy()
#         embedding = embedding / np.linalg.norm(embedding)
#     return embedding

# # ========== FACE DETECTION (placeholder) ==========
# def detect_faces(frame):
#     # Dummy HaarCascade — replace with YOLOv12 later
#     face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     boxes = face_cascade.detectMultiScale(gray, 1.1, 4)
#     return boxes  # (x, y, w, h)

# # ========== TRAIN CLASSIFIERS ==========
# def train_binary_classifiers(embedding_file=EMBEDDING_FILE, model_dir=CLASSIFIER_DIR):
#     os.makedirs(model_dir, exist_ok=True)
#     with open(embedding_file, 'rb') as f:
#         embeddings = pickle.load(f)

#     names = list(embeddings.keys())
#     X = np.array(list(embeddings.values()))
#     for i, name in enumerate(names):
#         y = np.array([1 if j == i else 0 for j in range(len(names))])
#         clf = LogisticRegression(max_iter=1000)
#         clf.fit(X, y)

#         model_path = os.path.join(model_dir, f"{name}_clf.pkl")
#         with open(model_path, 'wb') as f_out:
#             pickle.dump(clf, f_out)
#         print(f"[TRAINED] Classifier for {name} → {model_path}")

# # ========== FACE RECOGNITION ==========
# def recognize_face_binary(embedding, model_dir=CLASSIFIER_DIR, threshold=THRESHOLD):
#     best_match = "Unknown"
#     best_score = threshold

#     for fname in os.listdir(model_dir):
#         if fname.endswith('_clf.pkl'):
#             name = fname.replace('_clf.pkl', '')
#             with open(os.path.join(model_dir, fname), 'rb') as f:
#                 clf = pickle.load(f)
#             score = clf.predict_proba([embedding])[0][1]
#             if score > best_score:
#                 best_score = score
#                 best_match = name
#     return best_match

# # ========== OPTIONAL: RUN ONCE ==========
# # Uncomment this if you haven’t trained yet
# # train_binary_classifiers()

# # ========== REAL-TIME WEBCAM LOOP ==========
# cap = cv2.VideoCapture(0)
# print("Starting face recognition... Press 'q' to quit.")

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Mirror the frame
#     frame = cv2.flip(frame, 1)

#     boxes = detect_faces(frame)
#     for (x, y, w, h) in boxes:
#         # Expand box slightly
#         x1 = int(x - 0.02 * w)
#         y1 = int(y - 0.05 * h)
#         x2 = int(x + w + 0.02 * w)
#         y2 = int(y + h + 0.05 * h)
#         x1, y1 = max(0, x1), max(0, y1)
#         x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)

#         face_crop = frame[y1:y2, x1:x2]
#         embedding = get_face_embedding(face_crop)
#         name = recognize_face_binary(embedding)

#         # Display
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
#         cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

#     cv2.imshow("Face Recognition", frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()
